In [4]:
import tensorflow
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [7]:
import urllib
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7f7adc6cbbd0>)

In [47]:
pre_trained_model = InceptionV3(input_shape = (150,150,3),
                                 include_top = False,
                                 weights = None)
pre_trained_model.load_weights(weights_file)

In [48]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 74, 74, 32)  96          ['conv2d_188[0][0]']             
 Normalization)                                                                                   
                                                                                       

In [49]:
for layers in pre_trained_model.layers:
  layers.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')

In [50]:
last_output_shape = last_layer.output_shape  

In [51]:
last_output = last_layer.output

In [52]:
print(last_output_shape)
print(last_output)

(None, 7, 7, 768)
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")


In [53]:
X = tensorflow.keras.layers.Flatten()(last_layer.output)
X = tensorflow.keras.layers.Dense(units=1024,activation = tensorflow.nn.relu)(X)
X = tensorflow.keras.layers.Dense(units=1,activation = tensorflow.nn.sigmoid)(X)

In [54]:
from tensorflow.keras import Model
model = Model(pre_trained_model.input,X)

In [58]:
model.compile(optimizer = 'adam',loss= 'binary_crossentropy',metrics = ['accuracy'])

In [32]:
import urllib.request
import zipfile
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [56]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tr_datagen = ImageDataGenerator(rescale = 1/255)
tr_generator = tr_datagen.flow_from_directory('horse-or-human/training',target_size = (150,150),class_mode = 'binary')

Found 1027 images belonging to 2 classes.


In [41]:
tr_generator.data_format

'channels_last'

In [59]:
model.fit_generator(tr_generator,epochs = 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/5
33/33 [==============================] - 53s 1s/step - loss: 0.0730 - accuracy: 0.9757
Epoch 2/5
33/33 [==============================] - 49s 1s/step - loss: 0.0180 - accuracy: 0.9990
Epoch 3/5
33/33 [==============================] - 49s 1s/step - loss: 8.1972e-07 - accuracy: 1.0000
Epoch 4/5
33/33 [==============================] - 51s 2s/step - loss: 3.1554e-07 - accuracy: 1.0000
Epoch 5/5
33/33 [==============================] - 50s 2s/step - loss: 1.0735e-07 - accuracy: 1.0000


In [61]:
from tensorflow.keras.preprocessing import image

In [62]:
img = image.load_img('download.jpg',target_size=(150,150,3))

In [63]:
img = image.img_to_array(img)

In [65]:
img.shape

(150, 150, 3)

In [68]:
import numpy as np
x = np.expand_dims(img,axis=0)

In [69]:
x.shape

(1, 150, 150, 3)

In [70]:
x= np.vstack([x])

In [71]:
x.shape

(1, 150, 150, 3)

In [73]:
classification = model.predict(x)

In [74]:
if classification>0.5:
  print('horse')
else:
  print('human')

human


In [75]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 74, 74, 32)  96          ['conv2d_188[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [77]:
model.save('New_model')

INFO:tensorflow:Assets written to: New_model/assets


In [78]:
from google.colab import files

In [81]:
!zip -r model.zip New_model

  adding: New_model/ (stored 0%)
  adding: New_model/variables/ (stored 0%)
  adding: New_model/variables/variables.index (deflated 78%)
  adding: New_model/variables/variables.data-00000-of-00001 (deflated 16%)
  adding: New_model/saved_model.pb (deflated 92%)
  adding: New_model/keras_metadata.pb (deflated 96%)
  adding: New_model/assets/ (stored 0%)
